# Marketing Post Image Generator

This web app will help user to generator image in 3 steps.

    1. Upload an image containing a promotional object;
        segement the promotional object by covering it;
        The segmentation can be further improved by selecting points (in the object and in the background).
    2. Generate a image by giving the positive and negative prompts.
    3. Add texts of the company name, the slogan and the key words of the promotioanl object.

The generator use the facebook segment anything model to infer the segmentation task; Use stable diffusion 2--inpainting to generator the background image; Use grids like the rule of thirds to find where to add texts in the generated image.

Note:

  The prompt for image generation supports English and Chinese. To support more languages, just change the dropdown function in the web interface, eg "src_language = gr.Dropdown(choices=lang_codes, default='English', label='Prompt language')".
  
  For adding text in the image, the code can only display English and Chinese correctly. To support more languages, we need to install more fonts and modify the fonts used in the "addText" function.

## Deep Learning models set up

### Install

In [1]:
!pip install -q gradio
%load_ext gradio

!pip install transformers diffusers huggingface_hub xformers scipy ftfy accelerate controlnet_aux
# !pip install git+https://github.com/huggingface/diffusers
# !pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/facebookresearch/segment-anything.git


########## fonts install. The fonts are used to draw text in the image (tab of add text)
########## need to install founts for each language. As a example, below is the founts of chinese
##########
# !apt-get install fontconfig
# !apt-get install ttf-mscorefonts-installer
!wget "https://www.wfonts.com/download/data/2014/06/01/simhei/simhei.zip"
!unzip "simhei.zip"
!rm "simhei.zip"
!rm "sharefonts.net.txt"
!mv SimHei.ttf /usr/share/fonts/truetype/

!wget "https://www.wfonts.com/download/data/2014/06/01/fangsong/fangsong.zip"
!unzip "fangsong.zip"
!rm "fangsong.zip"
!rm "sharefonts.net.txt"
!mv chinese.simfang.ttf /usr/share/fonts/truetype/

!wget "https://www.wfonts.com/download/data/2016/07/15/kaiti/kaiti.zip"
!unzip "kaiti.zip"
!rm "kaiti.zip"
!rm "sharefonts.net.txt"
!mv simkai.ttf /usr/share/fonts/truetype/

!wget "https://www.wfonts.com/download/data/2014/06/01/lisu/lisu.zip"
!unzip "lisu.zip"
!rm "lisu.zip"
!rm "sharefonts.net.txt"
!mv chinese.simli.ttf /usr/share/fonts/truetype/

!fc-list :lang=zh


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:0

### Import

In [2]:
# segment anything
from segment_anything import build_sam, SamPredictor
import cv2
import numpy as np
import matplotlib.pyplot as plt

# diffusers
import PIL
import requests
import torch
from io import BytesIO
from diffusers import StableDiffusionInpaintPipeline

# translation
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# image process
from PIL import Image, ImageDraw, ImageFont, ImageColor
from torchvision.ops import masks_to_boxes

# Gradio (the web GUI)
import gradio as gr


### Load models

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load SAM model
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
sam_checkpoint = 'sam_vit_h_4b8939.pth'
sam_predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))

### Load Stable Diffusion -- Inpainting
sd_pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
).to(device)

### Load translation model -- nllb-200-distilled-600M
checkpoint ="facebook/nllb-200-distilled-600M"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
CN2EN_translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang='zho_Hans', tgt_lang='eng_Latn')

--2023-06-23 12:23:34--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.10, 13.227.219.70, 13.227.219.59, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth.1’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   233MB/s    in 12s     

2023-06-23 12:23:46 (208 MB/s) - ‘sam_vit_h_4b8939.pth.1’ saved [2564550879/2564550879]



Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

### Inference functions

In [5]:
## SAM segmentation by using points and box
def segment(image, points, labels, box, sam_model=sam_predictor):
  sam_model.set_image(image)
  if (labels is None or not labels.any()) and (box is None or not box.any()):
    return image
  elif labels is None or not labels.any():
    labels = None
    points = None
  elif box is None or not box.any():
    box = None
  masks, _, _ = sam_model.predict(
      point_coords = points,
      point_labels = labels,
      box = box,
      multimask_output = False,
      )
  return masks[0]


## stable diffusion--inpainting image by using prompt and mask
def inpaintImage(image, mask, prompt, negative_prompt, pipe=sd_pipe, seed=-1):
  # resize for inpainting
  w, h = image.size
  in_image = image.resize((512, 512))
  in_mask = mask.resize((512, 512))

  generator = torch.Generator(device).manual_seed(seed)

  result = pipe(image=in_image, mask_image=in_mask, prompt=prompt, negative_prompt=negative_prompt, generator=generator)
  result = result.images[0]
  return result.resize((w, h))


  ## translation
  def translate(src_lang, text):
    translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang=src_lang, tgt_lang='eng_Latn')
    output = translator(text, max_length=400)
    return output[0]['translation_text']

## Backend functions and values define

### Functions and values for image segmentation tab

In [6]:
def drawMask(mask, image, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.8])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)

    annotated_frame_pil = Image.fromarray(image).convert("RGBA")
    mask_image_pil = Image.fromarray((mask_image * 255).astype(np.uint8)).convert("RGBA")

    return np.array(Image.alpha_composite(annotated_frame_pil, mask_image_pil))

def predictSeg1(img):
  sam_predictor.set_image(img["image"])
  m_box = masks_to_boxes(torch.tensor([img["mask"][:,:,0]]))
  m_box = m_box.numpy()
  mask = segment(img["image"], None, None, m_box)
  img_with_mask = drawMask(mask, img["image"])
  mask_img = Image.fromarray(mask)
  return img_with_mask, mask_img, m_box

def predictSeg2(img, points, labels, box):
  sam_predictor.set_image(img["image"])
  mask = segment(img["image"], np.array(points), np.array(labels), box)
  img_with_mask = drawMask(mask, img["image"])
  mask_img = Image.fromarray(mask)
  return img_with_mask, mask_img

frame_point_size =[1, 10]

def plotPoints(img, points, labels):
  img_size=img.shape
  half_box_size=max(img_size[1]//100,1)
  border_width = half_box_size//3

  for i in range(len(labels)):
    x1=max(0,points[i][0]-half_box_size)
    y1=max(0,points[i][1]-half_box_size)
    x2=min(img_size[1],points[i][0]+half_box_size)
    y2=min(img_size[0],points[i][1]+half_box_size)
    if labels[i]==0:
      img[y1:y2, x1:x2] = np.array([255,0,0])
    else:
      img[y1:y2, x1:x2] = np.array([0,255,0])
    img[y1:y2, x1 : x1 + border_width] = np.array([200,200,200])
    img[y1:y2, x2 - border_width : x2] = np.array([200,200,200])
    img[y1 : y1 + border_width, x1:x2] = np.array([200,200,200])
    img[y2 - border_width : y2, x1:x2] = np.array([200,200,200])
  return img

def frameVisible(labels):
  if len(labels)>0:
    return gr.update(visible=True),gr.update(visible=True)
  else:
    return gr.update(visible=False),gr.update(visible=False)

def selectPoint(img, points, labels, choice, points_frame, evt: gr.SelectData):
  m_point= evt.index
  # points.append([m_point[1],m_point[0]])
  points.append(m_point)
  if choice == "object":
    labels.append(1)
  else:
    labels.append(0)
  points_frame[int((len(labels)-1)/frame_point_size[1])][(len(labels)-1)%frame_point_size[1]] = ("@" if labels[-1] == 1 else "X") + str(points[-1])
  points_img = plotPoints(img, points, labels)
  return [points_img, points, labels, points_frame]

def deletePoint(img, points, labels, points_frame, evt: gr.SelectData):
  delete_idx= evt.index[0]*frame_point_size[1]+evt.index[1]
  if delete_idx >= len(labels):
    points_img = plotPoints(img, points, labels)
    return [points_img, points, labels, points_frame]
  del points[delete_idx]
  del labels[delete_idx]
  for i in range(delete_idx, len(labels)):
    x = int(i/frame_point_size[1])
    y = i%frame_point_size[1]
    points_frame[x][y] = ("@" if labels[i] == 1 else "X") + str(points[i])
  points_frame[int(len(labels)/frame_point_size[1])][len(labels)%frame_point_size[1]] = ""
  points_img = plotPoints(img, points, labels)
  return [points_img, points, labels, points_frame]

def deleteAllPoint(img, points, labels, points_frame):
  for i in range(len(labels)):
    x = int(i/frame_point_size[1])
    y = i%frame_point_size[1]
    points_frame[x][y] = ""
  return [img, [], [], points_frame]

### Functions and values for image generation tab (stable diffusion)

In [7]:
codes_as_string = '''Acehnese (Arabic script)	ace_Arab
Acehnese (Latin script)	ace_Latn
Mesopotamian Arabic	acm_Arab
Ta’izzi-Adeni Arabic	acq_Arab
Tunisian Arabic	aeb_Arab
Afrikaans	afr_Latn
South Levantine Arabic	ajp_Arab
Akan	aka_Latn
Amharic	amh_Ethi
North Levantine Arabic	apc_Arab
Modern Standard Arabic	arb_Arab
Modern Standard Arabic (Romanized)	arb_Latn
Najdi Arabic	ars_Arab
Moroccan Arabic	ary_Arab
Egyptian Arabic	arz_Arab
Assamese	asm_Beng
Asturian	ast_Latn
Awadhi	awa_Deva
Central Aymara	ayr_Latn
South Azerbaijani	azb_Arab
North Azerbaijani	azj_Latn
Bashkir	bak_Cyrl
Bambara	bam_Latn
Balinese	ban_Latn
Belarusian	bel_Cyrl
Bemba	bem_Latn
Bengali	ben_Beng
Bhojpuri	bho_Deva
Banjar (Arabic script)	bjn_Arab
Banjar (Latin script)	bjn_Latn
Standard Tibetan	bod_Tibt
Bosnian	bos_Latn
Buginese	bug_Latn
Bulgarian	bul_Cyrl
Catalan	cat_Latn
Cebuano	ceb_Latn
Czech	ces_Latn
Chokwe	cjk_Latn
Central Kurdish	ckb_Arab
Crimean Tatar	crh_Latn
Welsh	cym_Latn
Danish	dan_Latn
German	deu_Latn
Southwestern Dinka	dik_Latn
Dyula	dyu_Latn
Dzongkha	dzo_Tibt
Greek	ell_Grek
English	eng_Latn
Esperanto	epo_Latn
Estonian	est_Latn
Basque	eus_Latn
Ewe	ewe_Latn
Faroese	fao_Latn
Fijian	fij_Latn
Finnish	fin_Latn
Fon	fon_Latn
French	fra_Latn
Friulian	fur_Latn
Nigerian Fulfulde	fuv_Latn
Scottish Gaelic	gla_Latn
Irish	gle_Latn
Galician	glg_Latn
Guarani	grn_Latn
Gujarati	guj_Gujr
Haitian Creole	hat_Latn
Hausa	hau_Latn
Hebrew	heb_Hebr
Hindi	hin_Deva
Chhattisgarhi	hne_Deva
Croatian	hrv_Latn
Hungarian	hun_Latn
Armenian	hye_Armn
Igbo	ibo_Latn
Ilocano	ilo_Latn
Indonesian	ind_Latn
Icelandic	isl_Latn
Italian	ita_Latn
Javanese	jav_Latn
Japanese	jpn_Jpan
Kabyle	kab_Latn
Jingpho	kac_Latn
Kamba	kam_Latn
Kannada	kan_Knda
Kashmiri (Arabic script)	kas_Arab
Kashmiri (Devanagari script)	kas_Deva
Georgian	kat_Geor
Central Kanuri (Arabic script)	knc_Arab
Central Kanuri (Latin script)	knc_Latn
Kazakh	kaz_Cyrl
Kabiyè	kbp_Latn
Kabuverdianu	kea_Latn
Khmer	khm_Khmr
Kikuyu	kik_Latn
Kinyarwanda	kin_Latn
Kyrgyz	kir_Cyrl
Kimbundu	kmb_Latn
Northern Kurdish	kmr_Latn
Kikongo	kon_Latn
Korean	kor_Hang
Lao	lao_Laoo
Ligurian	lij_Latn
Limburgish	lim_Latn
Lingala	lin_Latn
Lithuanian	lit_Latn
Lombard	lmo_Latn
Latgalian	ltg_Latn
Luxembourgish	ltz_Latn
Luba-Kasai	lua_Latn
Ganda	lug_Latn
Luo	luo_Latn
Mizo	lus_Latn
Standard Latvian	lvs_Latn
Magahi	mag_Deva
Maithili	mai_Deva
Malayalam	mal_Mlym
Marathi	mar_Deva
Minangkabau (Arabic script)	min_Arab
Minangkabau (Latin script)	min_Latn
Macedonian	mkd_Cyrl
Plateau Malagasy	plt_Latn
Maltese	mlt_Latn
Meitei (Bengali script)	mni_Beng
Halh Mongolian	khk_Cyrl
Mossi	mos_Latn
Maori	mri_Latn
Burmese	mya_Mymr
Dutch	nld_Latn
Norwegian Nynorsk	nno_Latn
Norwegian Bokmål	nob_Latn
Nepali	npi_Deva
Northern Sotho	nso_Latn
Nuer	nus_Latn
Nyanja	nya_Latn
Occitan	oci_Latn
West Central Oromo	gaz_Latn
Odia	ory_Orya
Pangasinan	pag_Latn
Eastern Panjabi	pan_Guru
Papiamento	pap_Latn
Western Persian	pes_Arab
Polish	pol_Latn
Portuguese	por_Latn
Dari	prs_Arab
Southern Pashto	pbt_Arab
Ayacucho Quechua	quy_Latn
Romanian	ron_Latn
Rundi	run_Latn
Russian	rus_Cyrl
Sango	sag_Latn
Sanskrit	san_Deva
Santali	sat_Olck
Sicilian	scn_Latn
Shan	shn_Mymr
Sinhala	sin_Sinh
Slovak	slk_Latn
Slovenian	slv_Latn
Samoan	smo_Latn
Shona	sna_Latn
Sindhi	snd_Arab
Somali	som_Latn
Southern Sotho	sot_Latn
Spanish	spa_Latn
Tosk Albanian	als_Latn
Sardinian	srd_Latn
Serbian	srp_Cyrl
Swati	ssw_Latn
Sundanese	sun_Latn
Swedish	swe_Latn
Swahili	swh_Latn
Silesian	szl_Latn
Tamil	tam_Taml
Tatar	tat_Cyrl
Telugu	tel_Telu
Tajik	tgk_Cyrl
Tagalog	tgl_Latn
Thai	tha_Thai
Tigrinya	tir_Ethi
Tamasheq (Latin script)	taq_Latn
Tamasheq (Tifinagh script)	taq_Tfng
Tok Pisin	tpi_Latn
Tswana	tsn_Latn
Tsonga	tso_Latn
Turkmen	tuk_Latn
Tumbuka	tum_Latn
Turkish	tur_Latn
Twi	twi_Latn
Central Atlas Tamazight	tzm_Tfng
Uyghur	uig_Arab
Ukrainian	ukr_Cyrl
Umbundu	umb_Latn
Urdu	urd_Arab
Northern Uzbek	uzn_Latn
Venetian	vec_Latn
Vietnamese	vie_Latn
Waray	war_Latn
Wolof	wol_Latn
Xhosa	xho_Latn
Eastern Yiddish	ydd_Hebr
Yoruba	yor_Latn
Yue Chinese	yue_Hant
Chinese (Simplified)	zho_Hans
Chinese (Traditional)	zho_Hant
Standard Malay	zsm_Latn
Zulu	zul_Latn'''

temp_codes = codes_as_string.split('\n')

flores_codes = {}
for code in temp_codes:
  lang, lang_code = code.split('\t')
  flores_codes[lang] = lang_code

lang_codes = list(flores_codes.keys())

In [8]:
def keepMaskImgOnly(mask, img):
    h, w = mask.shape[-2:]
    mask[mask>0]=1
    mask_image = mask * np.array([[[1,1,1]]])
    return Image.fromarray(np.multiply(img["image"], mask))


def generateImg(image, mask, pos_prompt, neg_prompt, src_language):
  if src_language == "English":
    en_pos_prompt = pos_prompt
    en_neg_prompt = neg_prompt
  elif src_language == "Chinese (Simplified)":
    en_pos_prompt = CN2EN_translator(pos_prompt, max_length=400)[0]['translation_text']
    en_neg_prompt = CN2EN_translator(neg_prompt, max_length=400)[0]['translation_text']
  else:
    src_lang = flores_codes[src_language]
    en_pos_prompt = translate(src_lang, pos_prompt)
    en_neg_prompt = translate(src_lang, neg_prompt)

  inverted_mask = ((255 - mask)).astype(np.uint8)
  # image_mask_pil = Image.fromarray(mask)
  inverted_image_mask_pil = Image.fromarray(inverted_mask)
  generated_image = inpaintImage(image=Image.fromarray(image), mask=inverted_image_mask_pil, prompt=en_pos_prompt, negative_prompt=en_neg_prompt)
  inverted_mask[inverted_mask>0]=1
  generated_image_without_object=inverted_mask*generated_image
  generated_image2 = image + generated_image_without_object
  m_box = masks_to_boxes(torch.tensor([mask[:,:,0]]))
  m_box = m_box.numpy()
  return generated_image, generated_image2, m_box

### Functions and values for adding text tab

In [9]:
def addText(img, texts, text_point, text_align):
  m_W, m_H = img.size
  img = img.convert("RGBA")
  draw = ImageDraw.Draw(img)

  fsize1 = m_W//20
  fsize2 = fsize1*2//5
  fsize3 = fsize1//2
  line_spacing1 = fsize1//10
  line_spacing2 = fsize1//5
  # font1 = ImageFont.truetype('LiberationSans-Bold', fsize1)
  # font2 = ImageFont.truetype('LiberationSansNarrow-Italic.ttf', fsize2)
  # font3 = ImageFont.truetype('LiberationMono-Regular.ttf', fsize3)
  font1 = ImageFont.truetype('SimHei.ttf', fsize1)
  font2 = ImageFont.truetype('chinese.simfang.ttf', fsize2)
  font3 = ImageFont.truetype('chinese.simli.ttf', fsize3)
  if text_align[1]>0:
    text_point[1]+=fsize1+fsize2+fsize3+line_spacing1+line_spacing2
  txt_layer = Image.new("RGBA", img.size, (255, 255, 255, 0))
  drawT = ImageDraw.Draw(txt_layer)
  if text_align[0]<0:
    drawT.text(text_point, texts[0], font=font1, fill=(0,255,255, 100))
    draw.text((text_point[0], text_point[1]+fsize1+line_spacing1), texts[1].decode('utf-8'), font=font2, fill='blue')
    draw.text((text_point[0], text_point[1]+fsize1+fsize2+line_spacing1+line_spacing2), texts[2], font=font3, fill='yellow')
  elif text_align[0]>0:
    textwidth, textheight = drawT.textsize(texts[0], font1)
    drawT.text((text_point[0]-textwidth, text_point[1]), texts[0], font=font1, fill=(0,255,255, 100))
    textwidth, textheight = draw.textsize(texts[1], font2)
    draw.text((text_point[0]-textwidth, text_point[1]+fsize1+line_spacing1), texts[1], font=font2, fill='blue')
    textwidth, textheight = draw.textsize(texts[2], font3)
    draw.text((text_point[0]-textwidth, text_point[1]+fsize1+fsize2+line_spacing1+line_spacing2), texts[2], font=font3, fill='yellow')
  else:
    textwidth, textheight = drawT.textsize(texts[0], font1)
    drawT.text((text_point[0]-textwidth/2, text_point[1]), texts[0], font=font1, fill=(0,255,255, 100))
    textwidth, textheight = draw.textsize(texts[1], font2)
    draw.text((text_point[0]-textwidth/2, text_point[1]+fsize1+line_spacing1), texts[1], font=font2, fill='blue')
    textwidth, textheight = draw.textsize(texts[2], font3)
    draw.text((text_point[0]-textwidth/2, text_point[1]+fsize1+fsize2+line_spacing1+line_spacing2), texts[2], font=font3, fill='yellow')
  # img.show()
  new_img = Image.alpha_composite(img, txt_layer)
  # new_img.show()
  return new_img
  # return np.asarray(new_img)

def arrange4(img, m_box, texts):
  m_W, m_H = img.size
  box_center = [ (m_box[2]+m_box[0])/m_W-1,  (m_box[3]+m_box[1])/m_H - 1]
  text_point=[0,0]
  text_align=[-1,0]
  if box_center[0]<=0:
    text_align[0]=1
    text_point[0]=m_W - m_box[0]
  else:
    text_point[0]=m_W - m_box[2]
  if box_center[1]<=0:
    text_align[1]=1
    text_point[-1]=m_H - m_box[1]
  else:
    text_point[-1]=m_H - m_box[3]
  result = addText(img, texts, text_point, text_align)
  return result

def arrange9(img, m_box, texts):
  m_W, m_H = img.size
  box_center = [ (m_box[2]+m_box[0])/m_W-1,  (m_box[3]+m_box[1])/m_H - 1]
  text_point=[0,0]
  text_align=[-1,0]
  if box_center[0]<=1/3 and box_center[0]>-1/3: # add text in the middle colum
    text_align[0]=0
    text_point[0]=m_W/2
    if box_center[1]<=0:
      text_align[1]=1
      text_point[-1]=m_H - m_box[1]
    else:
      text_point[-1]=m_H - m_box[3]
    result = addText(img, texts, text_point, text_align)
    return result
  elif box_center[1]<=1/3 and box_center[1]>-1/3: # add text in the middle row
    text_point[-1]=m_box[1]
    if box_center[0]<=0:
      text_align[0]=1
      text_point[0]=m_W - m_box[0]
    else:
      text_point[0]=m_W - m_box[2]
    result = addText(img, texts, text_point, text_align)
    return result
  else:
    return arrange4(img, m_box, texts)


def addText2Img(img, m_title, m_slogan, m_keywords, object_box):
  texts = [m_title, m_slogan, m_keywords]
  image = Image.fromarray(img)
  text_img = arrange9(image, object_box[0], texts)
  return text_img

## Web interface design by using Gradio

In [11]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
  gr.Markdown(
  """
  # Marketing Post Image Generator
  """)
  object_box = gr.State([])
  #############################################################
  with gr.Tab("1) Image segmentation"):
    gr.Markdown("Please upload a photo and cover the promotional object to segment it.")
    with gr.Row():
      inp_org = gr.ImageMask(label="Original")
      out_seg1 = gr.Image(interactive = False,label="Segmentation")
      mask_seg = gr.Image(interactive = False, label="mask", visible = False)
    run_seg1 = gr.Button("segment image")
    # gr.Examples(
    #     [
    #         "./sample1.jpg",
    #     ],
    #     inp_org
    # )
    gr.Markdown("To optimize the segmentation, Please click to add the points to indicate the object and the background.")
    m_radio = gr.Radio(
        ["object", "background"], label="Please select the type of point to click", value = "object",interactive = True
    )
    m_points = gr.State([])
    m_labels = gr.State([])
    m_box = gr.State([])
    points_frame = gr.Dataframe(value=[[""]*frame_point_size[1]] * frame_point_size[0], interactive=False, type="array", label="Click to delete the selected point", visible=False)
    delete_all_points = gr.Button("delete all  points", visible=False)
    with gr.Row():
      inp_seg2 = gr.Image(label="add points")
      out_seg2 = gr.Image(label="fine segmention")
    run_seg2 = gr.Button("fine segment")


  ############################################################
  with gr.Tab("2) Image generation"):
    # src_language = gr.Dropdown(choices=lang_codes, default='English', label='prompt language')
    src_language = gr.Dropdown(choices=['Chinese (Simplified)', 'English'], value='Chinese (Simplified)', label='prompt language')
    pos_prompt = gr.Textbox(label="positive prompt")
    neg_prompt = gr.Textbox(label="negative prompt")
    run_GC = gr.Button("generate image")
    with gr.Row():
      inp_GC = gr.Image(interactive=False,label="object")
      out_GC = gr.Image(interactive = False,label="generation")

  ############################################################
  with gr.Tab("3) Adding texts"):
    m_title = gr.Textbox(label="The company name")
    m_slogan = gr.Textbox(label="The slogan")
    m_keywords = gr.Textbox(label="The product key words")
    add_text = gr.Button("add texts")
    with gr.Row():
      without_text = gr.Image(interactive=False,label="without texts")
      with_text = gr.Image(interactive = False,label="with texts")
  object_box = gr.State([])

  ############################################################
  #### GUI interactive trigers

  ## Tab("Image segmentation")
  inp_seg2.select(selectPoint, [out_seg1, m_points, m_labels, m_radio, points_frame], [inp_seg2, m_points, m_labels, points_frame]).then(frameVisible,m_labels,[points_frame,delete_all_points])
  points_frame.select(deletePoint, [out_seg1, m_points, m_labels, points_frame], [inp_seg2, m_points, m_labels, points_frame]).then(frameVisible,m_labels,[points_frame,delete_all_points])
  # run_seg1.click(predictSegtest, inp_org, prompt)
  run_seg1.click(predictSeg1, inp_org, [out_seg1, mask_seg, m_box]).then(deleteAllPoint, [out_seg1, m_points, m_labels, points_frame], [inp_seg2, m_points, m_labels, points_frame]).then(frameVisible,m_labels,[points_frame,delete_all_points]).then(keepMaskImgOnly, [mask_seg, inp_org], inp_GC)
  run_seg2.click(predictSeg2, [inp_org, m_points, m_labels, m_box], [out_seg2, mask_seg]).then(keepMaskImgOnly, [mask_seg, inp_org], inp_GC)
  delete_all_points.click(deleteAllPoint, [out_seg1, m_points, m_labels, points_frame], [inp_seg2, m_points, m_labels, points_frame]).then(frameVisible,m_labels,[points_frame,delete_all_points])

  ## Tab("Image generation")
  run_GC.click(generateImg, [inp_GC, mask_seg, pos_prompt, neg_prompt, src_language],[out_GC, without_text, object_box])

  ## Tab("Adding texts")
  add_text.click(addText2Img, [without_text,  m_title, m_slogan, m_keywords, object_box], with_text)


demo.launch(share=False)
# demo.launch(share=True, auth=("admin", "admin"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>